In [ ]:
!pip install -q bitsandbytes
!pip install -q datasets loralib sentencepiece
!pip install -q accelerate
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q git+https://github.com/huggingface/peft.git@4fd374e80d670781c0d82c96ce94d1215ff23306 --force-reinstall --no-deps
!pip install -q git+https://github.com/zifei-stanford/alpaca-lora.git
!git clone https://github.com/zifei-stanford/alpaca-lora.git
!pip install fire

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive',force_remount=True)

# import 

In [ ]:
import torch
import numpy as np
from peft import PeftModel,prepare_model_for_int8_training
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig
from datasets import Dataset

import transformers

%cd alpaca-lora
from utils.prompter import Prompter

torch.cuda.is_available()

In [ ]:
# !mkdir gdrive/MyDrive/cs221/output
!ls /content/gdrive/MyDrive/cs221/output/

In [ ]:
output_dir = '/content/gdrive/MyDrive/cs221/output/'

In [ ]:
torch.cuda.is_available()

In [ ]:
generation_config = GenerationConfig(
    # temperature=0.1,
    temperature=0, ### to make results reproducible, and self-consistent in self-referential QA
    top_p=0.75,
    num_beams=4,
)

def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

def evaluate(instruction, input=None, model=None):
    print('evaluating...')
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=256
        )
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        # print("Response:", output.split("### Response:")[1].strip())
        print(output)

def predict(model,input,instruction="short answer to question"):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )
    s = generation_output.sequences[0]
    # remove last eos token
    output = tokenizer.decode(s[:-1])
    #print("Response:", output.split("### Response:")[1].strip())
    return output.split("### Response:")[1].strip()
        

# load model

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained("yahma/llama-7b-hf")
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"  # Allow batched inference
model = LlamaForCausalLM.from_pretrained(
    "yahma/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
# model = PeftModel.from_pretrained(model, output_dir)

In [ ]:
model = PeftModel.from_pretrained(model, 'yahma/alpaca-7b-lora')

In [ ]:
### alternatively, load our own model
# model = PeftModel.from_pretrained(model, <path to saved model>)
model = PeftModel.from_pretrained(model, output_dir + 'baseline2')

In [ ]:
evaluate('short answer to question','what is my name',model=model)

# cook dataset

In [ ]:
import json
from datasets import Dataset
def generate_dataset(model,dir):
  from datasets import load_dataset
  import random
  dataset = load_dataset("nq_open")
  train_qs = dataset['train'][:500]['question']
  test_qs = dataset['validation'][:50]['question']
  val_qs = dataset['validation'][50:100]['question']
  def process_data(model,qs):
    data_ans = []
    # qs = qs[:2] ### for quick debugging
    for q in qs:
      print(len(qs))
      data_ans.append(predict(model,q))
    data_self = []
    for i in range(len(qs)):
      wrong_ans = random.choice(list(range(0,i))+list(range(i+1,len(qs))))
      dict_item = {}
      dict_item['input'] = "Would I have answered "+data_ans[i]+" if asked "+qs[i]
      dict_item['output'] ="Yes"
      data_self.append(dict_item)
      dict_item = {}
      dict_item['input'] = "Would I have answered "+data_ans[wrong_ans]+" if asked "+qs[i]
      dict_item['output'] ="No"
      data_self.append(dict_item)
    return (data_self,qs,data_ans)
  # train_data,train_qs,train_ans = process_data(model,train_qs)
  # # Add original question answer pairs to training data
  # for question,answer in zip(train_qs,train_ans):
  #   dict_item = {}
  #   dict_item['input'] = question
  #   dict_item['output'] = answer
  #   train_data.append(dict_item)
  # test_data = process_data(model,test_qs)[0]
  # val_data = process_data(model,val_qs)[0]
  val_self,val_qs,val_ans = process_data(model,val_qs)
  verif_data = []
  for question,answer in zip(val_qs,val_ans):
    dict_item={}
    dict_item['input']= question
    dict_item['output']=answer
    verif_data.append(dict_item)

  # with open(dir+'train.json','w') as outfile:
  #   json.dump(train_data,outfile)
  # with open(dir+'test.json','w') as outfile:
  #   json.dump(test_data,outfile)
  # with open(dir+'val.json','w') as outfile:
  #   json.dump(val_data,outfile)
  with open(dir+'verif.json','w') as outfile:
    json.dump(verif_data,outfile)

In [ ]:
generate_dataset(model,output_dir+'testrun/')


In [ ]:
from datasets import load_dataset
data = load_dataset("json", data_files=output_dir+'testrun/train.json')

In [ ]:
data['train']['input']

# finetune model

In [ ]:
### load data prepared from previous section
from datasets import load_dataset
data_path = output_dir+'testrun/train.json'
prompter = Prompter('alpaca')
cutoff_len = 256
def tokenize(prompt, add_eos_token=True):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=cutoff_len,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < cutoff_len
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    result["labels"] = result["input_ids"].copy()

    return result
def generate_and_tokenize_prompt(data_point):
    full_prompt = prompter.generate_prompt(
        "short answer to question",
        data_point["input"],
        data_point["output"],
    )
    tokenized_full_prompt = tokenize(full_prompt)
    return tokenized_full_prompt
data = load_dataset("json", data_files=data_path)['train']
train_data = data.map(generate_and_tokenize_prompt)

In [ ]:
model = prepare_model_for_int8_training(model)
for name, param in model.named_parameters():
  if 'lora' in name:
    param.requires_grad=True
model.print_trainable_parameters()

In [ ]:
trainer = transformers.Trainer(
    model=model, 
    train_dataset=train_data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=16, 
        gradient_accumulation_steps=1,
        warmup_steps=100, 
        num_train_epochs = 3,
        # max_steps=10, 
        learning_rate=1e-6, 
        fp16=True,
        logging_steps=10, 
        optim='adamw_torch',
        save_strategy='no',
        # save_steps=200,
        # save_total_limit=1,
        output_dir=output_dir
    ),
    data_collator=transformers.DataCollatorForSeq2Seq(
            tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
        )
)


In [ ]:
model.config.use_cache = False

In [ ]:
trainer.train()

# save model

In [ ]:
model.to('cpu')
model.save_pretrained(output_dir + 'baseline1') ### simply finetune on self-referential question

# quick checks

In [ ]:
model.cuda()

In [ ]:
### check some simple questinons
evaluate('What is my name', model=model)

# evaluate on test data

In [ ]:
from datasets import load_dataset
data = load_dataset("json", data_files=output_dir+'testrun/test.json')['train']

In [ ]:
data

In [ ]:
data[0]

In [ ]:
import csv
import numpy as np
def test_accuracy(model, filename=output_dir+'testrun/test.json'):
  correct_pos,correct_neg,num_pos,num_neg = 0,0,0,0
  # data = np.loadtxt(dir,delimiter="~ ", dtype=str)
  data = load_dataset("json", data_files=filename)['train']
  for row in data:
    input = row['input']
    true_response = row['output']
    predicted_response = predict(model,input)
    print(predicted_response)
    if true_response == "Yes":
      num_pos+=1
      if true_response==predicted_response[:3]:
        correct_pos+=1
    elif true_response == "No":
      num_neg+=1
      if true_response==predicted_response[:2]:
        correct_neg+=1

  print("Accuracy: ", float(correct_pos+correct_neg)/(num_pos+num_neg))
  print("Recall for positive: ", float(correct_pos)/num_pos)
  print("Recall for negative: ", float(correct_neg)/num_neg)
  return float(correct_pos+correct_neg)/len(data)

In [ ]:
test_accuracy(model)

In [ ]:
import re
def oracle_accuracy(model,filename=output_dir+'testrun/test.json'):
  correct_pos,correct_neg,num_pos,num_neg = 0,0,0,0
  data = load_dataset("json", data_files=filename)['train']
  for row in data:
    input = row['input']
    true_response = row['output']
    answer = re.findall("Would I have answered .* if asked",input)#22
    if len(answer)>0:
      answer = answer[0].strip()[22:-9]
    question = re.findall("if asked .*",input)
    if len(question)>0:
      question = question[0].strip()[9:]
    predicted_answer = predict(model,question)
    if true_response == "Yes":
      num_pos+=1
      if answer==predicted_answer:
        correct_pos+=1
    elif true_response == "No":
      num_neg+=1
      if answer!=predicted_answer:
        correct_neg+=1
  print("Accuracy: ", float(correct_pos+correct_neg)/(num_pos+num_neg))
  print("Recall for positive: ", float(correct_pos)/num_pos)
  print("Recall for negative: ", float(correct_neg)/num_neg)
  return float(correct_pos+correct_neg)/len(data)

In [ ]:
oracle_accuracy(model)

In [ ]:
def consistency_accuracy(model,filename=output_dir+'testrun/verif.json'):
  correct,total = 0,0
  data = load_dataset("json", data_files=filename)['train']
  for row in data:
    input = row['input']
    true_response = row['output']
    predicted_answer = predict(model,input)
    print(true_response)
    print(predicted_answer)
    if predicted_answer == true_response:
      correct+=1
    total+=1
  return float(correct)/total


In [ ]:
consistency_accuracy(model)